In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import os
from lips import get_root_path
# from lips.benchmark.airfransBenchmark import AirfRANSBenchmark
from lips.dataset.airfransDataSet import AirfRANSDataSet
import numpy as np

In [2]:
LIPS_PATH = get_root_path()
DIRECTORY_NAME = 'Dataset'
BENCHMARK_NAME = "DEFAULT"
LOG_PATH = LIPS_PATH + "lips_logs.log"
BENCH_CONFIG_PATH = os.path.join("airfoilConfigurations","benchmarks","confAirfoil.ini") 

directory_name='Dataset'

Similar model to the one in MLP_test_model.ipynb. However, here we have 128 nodes in hidden layer and using ReLU instead Tanh.

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(7, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 4)
        )
    
    def forward(self, x):
        return self.model(x)

In [14]:
def train_model(model, train, test, epochs=50):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    device = torch.device("cpu")
    model.to(device)
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in train:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # model.eval()
        # val_loss = 0.0
        # with torch.no_grad():
        #     for X_val, y_val in test:
        #         X_val, y_val = X_val.to(device), y_val.to(device)
        #         y_pred = model(X_val)
        #         loss = criterion(y_pred, y_val)
        #         val_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs} | Loss: {train_loss/len(train):.4f}")
        # print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss/len(test):.4f} | Test Loss: {val_loss/len(test):.4f}")
    
    return model

In [6]:
def load_data(dataset):
    # Deifinition of Input and Output attributes
    X_attr = ["x-position", "y-position", "x-inlet_velocity", "y-inlet_velocity", "x-normals", "y-normals", "distance_function"]
    y_attr = ["x-velocity", "y-velocity", "pressure", "turbulent_viscosity"]
    X, y = [], []
    
    # Looping through 10 items from the dataset to get the attributes
    for i in range(1,10):
        X.append(np.column_stack([dataset.data[attr][i] for attr in X_attr]))
        y.append(np.column_stack([dataset.data[attr][i] for attr in y_attr]))

    # Stacking the data to form a single matrix
    X = np.vstack(X)
    y = np.vstack(y)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
    X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)
    
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)
    
    return train_loader, test_loader

In [7]:
attr_names = (
        'x-position',
        'y-position',
        'x-inlet_velocity', 
        'y-inlet_velocity', 
        'distance_function', 
        'x-normals', 
        'y-normals', 
        'x-velocity', 
        'y-velocity', 
        'pressure', 
        'turbulent_viscosity',
    )
attr_x = attr_names[:7]
attr_y = attr_names[7:]

Taken from 2-Import_Airfoil_design_Dataset.ipynb

In [8]:
configuration_file = None #Convenient alternative but not required at this point
dataset_name = "my_dataset_test"
usecase_task = "scarce" #Four task are supported: 'full', 'scarce', 'reynolds', 'aoa'
usecase_split = "training" #Describe which data subset within a task to be used, the other option is testing
log_path = "dataset_log"

In [9]:
my_dataset = AirfRANSDataSet(config = configuration_file, 
                             name = dataset_name,
                             task = usecase_task,
                             split = usecase_split,
                             attr_names = attr_names, 
                             attr_x = attr_x, 
                             attr_y = attr_y, 
                             log_path = log_path)

In [10]:
my_dataset.load(path=DIRECTORY_NAME)

Loading dataset (task: scarce, split: train): 100%|██████████| 200/200 [00:43<00:00,  4.59it/s]


In [11]:
train, test = load_data(my_dataset)

In [15]:
model = MLP()

In [16]:
trained_model = train_model(model, train, test)

Epoch 1/50 | Loss: 344.7481
Epoch 2/50 | Loss: 324.4673
Epoch 3/50 | Loss: 306.4139
Epoch 4/50 | Loss: 290.2108
Epoch 5/50 | Loss: 274.3653
Epoch 6/50 | Loss: 257.5312
Epoch 7/50 | Loss: 240.0352
Epoch 8/50 | Loss: 221.7873
Epoch 9/50 | Loss: 202.7572
Epoch 10/50 | Loss: 183.2710
Epoch 11/50 | Loss: 163.7333
Epoch 12/50 | Loss: 143.9605
Epoch 13/50 | Loss: 124.3269
Epoch 14/50 | Loss: 104.9135
Epoch 15/50 | Loss: 86.1161
Epoch 16/50 | Loss: 68.4313
Epoch 17/50 | Loss: 52.3972
Epoch 18/50 | Loss: 38.3190
Epoch 19/50 | Loss: 26.7806
Epoch 20/50 | Loss: 17.9932
Epoch 21/50 | Loss: 12.1607
Epoch 22/50 | Loss: 9.2318
Epoch 23/50 | Loss: 8.8208
Epoch 24/50 | Loss: 10.1630
Epoch 25/50 | Loss: 12.2527
Epoch 26/50 | Loss: 14.1637
Epoch 27/50 | Loss: 15.3028
Epoch 28/50 | Loss: 15.4447
Epoch 29/50 | Loss: 14.7049
Epoch 30/50 | Loss: 13.3403
Epoch 31/50 | Loss: 11.6280
Epoch 32/50 | Loss: 9.8306
Epoch 33/50 | Loss: 8.1594
Epoch 34/50 | Loss: 6.7667
Epoch 35/50 | Loss: 5.6706
Epoch 36/50 | Loss: 4